## ATIVIDADE 2 - Web Scraping no site IMDB

Importações de **bibliotecas** utilizadas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import selenium
import pandas as pd
import json

Definição de qual **driver** será utilizado. Se necessário, mude conforme seu navegador.

In [2]:
# driver = webdriver.Chrome()
driver = webdriver.Firefox()

## Entrando na Página

Utilizaremos o driver para acessarmos a página do **IMDB**, acessar seu *menu* e assim chegar no **Calendário de lançamentos**

In [3]:
driver.get("https://www.imdb.com")

In [4]:
menu = driver.find_element(By.XPATH,r'//*[@id="imdbHeader-navDrawerOpen"]') 
menu.click()

In [7]:
calendario = driver.find_element(By.LINK_TEXT, r'Calendário de lançamento')

In [8]:
url = calendario.get_attribute("href")
driver.get(url)

## Pegando os Elementos

Agora, dentro da página escolhida, pegaremos os valores que precisamos

In [9]:
tag = driver.find_elements(By.CLASS_NAME, 'ipc-metadata-list-summary-item')

Listando todos os filmes presentes dentro da tag que contém os mesmos

In [10]:
for filme in tag:
    print(filme.find_element(By.CLASS_NAME, 'ipc-metadata-list-summary-item__t').text)

Longlegs: Vínculo Mortal (2024)
Afraid (2024)
Sting: Aranha Assassina (2024)
A Caça (2024)
Estômago II: O Poderoso Chef (2024)
Cidade; Campo (2024)
Os Fantasmas Ainda se Divertem - Beetlejuice Beetlejuice (2024)
Kill - O Massacre no Trem (2023)
O Menino e o Mestre (2023)
Corrida Maluca (2023)
Vovó Ninja (2024)
Não Fale o Mal (2024)
Transformers: O Início (2024)
Meu Amigo Pinguim (2024)
O Bastardo (2023)
Ligação Sombria (2023)
Silvio (2024)
Un silence (2023)
Meu Casulo de Drywall (2023)
A Forja: O Poder da Transformação (2024)
Hellboy e o Homem Torto (2024)
Golpe de Sorte em Paris (2023)
Passagrana (2024)
Saudade fez Morada aqui Dentro (2022)
Robô Selvagem (2024)
Coringa: Delírio a Dois (2024)
Não Solte! (2024)
Infestação (2023)
A Garota da Vez (2023)
Pássaro Branco: Uma História de Extraordinário (2023)
A Menina e o Dragão (2024)
Força Bruta: Sem Saída (2023)
Tudo Por Um Pop Star 2 (2024)
Sorria 2 (2024)
O Aprendiz (2024)
Super/Man: A História de Christopher Reeve (2024)
Perfekta: Uma 

### Definindo Funções

Funções tanto para extrair as informações dos filmes - *nome, data de lançamento, gênero e link de sua página* - e também do cast que está envolvido - *diretor e atores principais*

In [11]:
def extract_titulos(filme):
    nome = filme.find_element(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
    return nome.text

def extract_data_lancamento(sessao):
    nome = sessao.find_element(By.CSS_SELECTOR, 'h3.ipc-title__text')
    return nome.text

def extract_generos(filme):
    generos = filme.find_elements(By.CSS_SELECTOR, 'ul.ipc-inline-list--show-dividers.ipc-metadata-list-summary-item__tl li')
    return [g.text for g in generos]

def extract_links(filme, lista):
    nome = filme.find_element(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
    link = nome.get_attribute('href')
    lista.append(link)
    
    return link

In [12]:
def extract_filmes_info():
    global filmes_links
    filmes_links = []
    filmes_lista = []
    
    sections = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="calendar-section"]')
    
    for section in sections:
        data = extract_data_lancamento(section)
        
        filmes = section.find_elements(By.CSS_SELECTOR, 'li[data-testid="coming-soon-entry"]')
        
        for filme in filmes:
            filme_info = {}
            filme_info['Título'] = extract_titulos(filme)
            filme_info['Link'] = extract_links(filme, filmes_links)
            filme_info['Gêneros'] = extract_generos(filme)
            filme_info['Data de lançamento'] = data
            
            filmes_lista.append(filme_info)
    
    return filmes_lista

In [13]:
def extract_ator_diretor(aux, link):
    driver.get(link)

    diretor = driver.find_element(By.CSS_SELECTOR, '.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    ator = driver.find_elements(By.CSS_SELECTOR, 'a[data-testid="title-cast-item__actor"]')
    
    atores = [a.text for a in ator]

    aux.append(diretor.text)
    aux.append(atores)

    return aux

### Chamando as Funções e Salvando em um JSON

In [14]:
filmes = extract_filmes_info()   

df_filmes = pd.DataFrame(filmes)
display(df_filmes)

,Título,Link,Gêneros,Data de lançamento
0,Longlegs: Vínculo Mortal (2024),https://www.imdb.com/title/tt23468450/?ref_=rlm,"[Policial, Terror, Suspense]",29 de ago. de 2024
1,Afraid (2024),https://www.imdb.com/title/tt24577462/?ref_=rlm,"[Terror, Mistério, Ficção científica]",29 de ago. de 2024
2,Sting: Aranha Assassina (2024),https://www.imdb.com/title/tt20112746/?ref_=rlm,"[Terror, Ficção científica, Suspense]",29 de ago. de 2024
3,A Caça (2024),https://www.imdb.com/title/tt22493538/?ref_=rlm,"[Terror, Suspense]",29 de ago. de 2024
4,Estômago II: O Poderoso Chef (2024),https://www.imdb.com/title/tt28281591/?ref_=rlm,"[Comédia, Policial, Drama]",29 de ago. de 2024
...,...,...,...,...
77,Thunderbolts* (2025),https://www.imdb.com/title/tt20969586/?ref_=rlm,"[Ação, Aventura, Policial]",02 de mai. de 2025
78,Missão: Impossível 8 (2025),https://www.imdb.com/title/tt9603208/?ref_=rlm,"[Ação, Aventura, Suspense]",22 de mai. de 2025
79,Karate Kid (2025),https://www.imdb.com/title/tt1674782/?ref_=rlm,"[Ação, Drama, Família]",30 de mai. de 2025
80,F1 (2025),https://www.imdb.com/title/tt16311594/?ref_=rlm,"[Drama, Esportes]",26 de jun. de 2025


In [16]:
info = []
for link in filmes_links:
    aux = []
    info.append(extract_ator_diretor(aux, link))
    
    if (len(info) == 82):
        break

In [17]:
df_cast = pd.DataFrame(info)
display(df_cast)

,0,1
0,Osgood Perkins,"[Maika Monroe, Nicolas Cage, Blair Underwood, ..."
1,Chris Weitz,"[John Cho, Katherine Waterston, Keith Carradin..."
2,Kiah Roache-Turner,"[Noni Hazlehurst, Jermaine Fowler, Alyla Brown..."
3,Louis Lagayette,"[Lily Banda, Alec Newman, Kostis Kallivretakis..."
4,Marcos Jorge,"[Alessandro Andolfo, Giulio Beranek, Francesco..."
...,...,...
77,Jake Schreier,"[Florence Pugh, Lewis Pullman, Olga Kurylenko,..."
78,Christopher McQuarrie,"[Tom Cruise, Vanessa Kirby, Katy O'Brian, Hayl..."
79,Jonathan Entwistle,"[Joshua Jackson, Jackie Chan, Ralph Macchio, M..."
80,Joseph Kosinski,"[Brad Pitt, Kerry Condon, Tobias Menzies, Javi..."


### Transformando em JSON

Agora, combinaremos as duas listas em somente um dicionário e colocaremos ele dentro de um JSON

In [18]:
df_filmes['Diretor'] = df_cast[0].values
df_filmes['Atores'] = df_cast[1].values

In [21]:
filmes_dict = df_filmes.to_dict(orient='records')
filmes_json = json.dumps(filmes_dict, indent=4, ensure_ascii=False)

with open('Filmes.json', 'w', encoding='utf-8') as arquivo:
    arquivo.write(filmes_json)